# Majority vote labeling

In [80]:
from label_legends.preprocess import load_data, transform, majority
from collections import Counter
import polars as pl

ImportError: cannot import name 'majority' from 'label_legends.preprocess' (/Users/retipeter/Documents/TU Wien/NLP/nlp-ie-label-legends/src/label_legends/preprocess.py)

In [75]:
df1 = load_data().collect()
df2 = transform(df1)
df2 = df2.with_columns(df1.select("rewire_id"))
df2 = df2.with_columns(
    pl.col("rewire_id").str.extract(r"(\d+)$", 1).cast(pl.Int64).alias("rewire_id_number")
)
set_label = df2.group_by("rewire_id_number").agg([
    pl.col("label").sum().alias("label_sum"),
    pl.col("text").first().alias("text"),
    pl.col("tokens").first().alias("tokens"),
    pl.col("token_ids").first().alias("token_ids")
    ]
)
set_label = set_label.with_columns(
    pl.when(pl.col("label_sum") >= 2).then(1).otherwise(0).alias("label")  # Conditional update for 'label'
)
set_label

rewire_id_number,label_sum,text,tokens,token_ids,label
i64,i64,str,list[str],list[i64],i32
10784,1,"""♫ #NowPlaying ♫ > Marina and T…","[""♫"", ""#"", … ""gaogirl""]","[3000, 7, … 3000]",0
8021,2,"""She could just be an A) attent…","[""just"", "")"", … "".""]","[1483, 17, … 26]",1
14422,0,"""/u/xQueen-Bx Atlas, you need t…","[""/u/xqueen"", ""-"", … ""!""]","[3000, 23, … 0]",0
17045,0,"""Yet it isn't over for abusers …","[""abuser"", ""serial"", … "".""]","[116, 2369, … 26]",0
11022,0,"""🚨 IMPORTANT MESSAGE 🚨 Higher P…","[""🚨"", ""important"", … ""unitethewhite""]","[3000, 1363, … 3000]",0
…,…,…,…,…,…
6523,0,"""I’m losing weight to make girl…","[""lose"", ""weight"", … ""pretty""]","[1612, 2904, … 2071]",0
17798,1,"""She sounds insecure , Sounds l…","[""sound"", ""insecure"", … "".""]","[2504, 1400, … 26]",0
5633,0,"""what did she say? you remind m…","[""say"", ""?"", … ""?""]","[2331, 98, … 98]",0


In [76]:
majority

NameError: name 'majority' is not defined